MongoDB Local Connector: Splittet große GeoJson Data in mehrere Insertions in MongoDB

In [1]:
from pymongo import MongoClient
import json

def connect_to_local_mongodb():
    """
    Establishes a connection to a local MongoDB instance and returns the client and collection objects.
    """
    # Connect to the local MongoDB instance (assuming it's running on the default port 27017)
    client = MongoClient('localhost', 27017)

    # Access the specified MongoDB database
    db = client['ZIP_Polygon_Ger_Test']  # Replace 'your_local_database_name' with your desired local database name

    print("Connected to local MongoDB")

    # Specify the MongoDB collection name
    collection_name = 'ZIP_Polygon_Local_Ger_Test'

    # Create or access the 'ZIP_Polygon' collection
    zip_polygon_collection = db[collection_name]
    print(f"Collection '{collection_name}' created or accessed.")

    return client, zip_polygon_collection

def insert_geojson_data_into_local_mongodb(collection, geojson_file_path, chunk_size=1000):
    """
    Inserts GeoJSON data into the specified local MongoDB collection in chunks.
    """
    with open(geojson_file_path, 'r') as file:
        # Load GeoJSON data from the file
        geojson_data = json.load(file)

    # Split the GeoJSON features into chunks
    features = geojson_data['features']
    for i in range(0, len(features), chunk_size):
        chunk = features[i:i+chunk_size]
        # Insert the chunk into the specified collection
        collection.insert_many(chunk)
        print(f"Inserted {len(chunk)} features into local MongoDB collection '{collection.name}'.")

# Example usage of the function with a chunk size of 1000 features
mongo_client, zip_polygon_collection = connect_to_local_mongodb()
geojson_file_path = 'Germany_postal_code_polygons.json'  # Update the file path accordingly
insert_geojson_data_into_local_mongodb(zip_polygon_collection, geojson_file_path, chunk_size=1000)

# Close the local MongoDB connection when done
mongo_client.close()


Connected to local MongoDB
Collection 'ZIP_Polygon_Local_Ger_Test' created or accessed.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 1000 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.
Inserted 264 features into local MongoDB collection 'ZIP_Polygon_Local_Ger_Test'.


In [11]:
from pymongo import MongoClient
import json
import os
import geojson
import osmnx as ox
import pandas as pd

def connect_to_local_mongodb(database_name, collection_name):
    """
    Establishes a connection to a local MongoDB instance and returns the client and collection objects.
    """
    client = MongoClient('localhost', 27017)
    db = client[database_name]
    target_collection = db[collection_name]
    return client, target_collection

def insert_geojson_data_into_mongodb(collection, geojson_data, chunk_size=1000):
    """
    Inserts GeoJSON data into the specified MongoDB collection in chunks.
    """
    # Transform the data into a list of dictionaries
    mongo_data = [{'postal_code': feature['properties'].get('postal_code', None), 'polygon': feature['geometry']} for feature in geojson_data['features']]

    # Split the GeoJSON features into chunks
    for i in range(0, len(mongo_data), chunk_size):
        chunk = mongo_data[i:i + chunk_size]
        # Insert the chunk into the specified collection
        collection.insert_many(chunk)
        print(f"Inserted {len(chunk)} documents into MongoDB collection '{collection.name}'.")

# User input for the city or country name
#place_input = input("Geben Sie den Stadtnamen oder Ländernamen ein: ")

# Fetch postal code polygons using OSMnx
place_postal_code_polygons = ox.features_from_place(place_input, tags={"boundary": "postal_code"})
df = pd.DataFrame(place_postal_code_polygons).dropna(subset=['postal_code', 'geometry'])

# Create GeoJSON features from the input data
geojson_features = [geojson.Feature(geometry=feature['geometry'], properties={'postal_code': feature['postal_code']}) for _, feature in df.iterrows()]

# Create a GeoJSON FeatureCollection
geojson_data = geojson.FeatureCollection(geojson_features)

# Example call for user input
file_name = "germany_geojson_data.geojson"

# Determine the script's directory
script_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()

# Create the full path for the GeoJSON file
geojson_file_path = os.path.join(script_dir, file_name)

# Example usage of the function with a chunk size of 1000 features
database_name = 'ZIP_Polygon_Ger_Test'  # Update the database name accordingly
collection_name = 'ZIP_Polygon_Local_Ger_Test_'  # Using place_input for collection name
mongo_client, zip_polygon_collection = connect_to_local_mongodb(database_name, collection_name)
insert_geojson_data_into_mongodb(zip_polygon_collection, geojson_data, chunk_size=1000)

# Close the MongoDB connection when done
mongo_client.close()


InsufficientResponseError: Nominatim geocoder returned 0 results for query ''